In [ ]:
import pandas as p
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
x = np.load('/Users/coral/Desktop/Fold 1/images/fold1/images.npy')
y = np.load('/Users/coral/Desktop/Fold 1/images/fold1/types.npy')

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(2124, 256, 256, 3) (2124,)
(532, 256, 256, 3) (532,)


In [ ]:
#encoding y_train
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

y_train = onehot_encoded

D:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#encoding y_test
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_test)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

y_test = onehot_encoded

D:\Anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state= 0)

In [ ]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(1911, 256, 256, 3) (1911, 19)
(213, 256, 256, 3) (213, 19)
(532, 256, 256, 3) (532, 19)


## CNN

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score

# Define the model
model_cnn = Sequential()
model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(64, activation='relu'))
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(19, activation='softmax'))


model_cnn.compile(optimizer=SGD(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Data conversion
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Train the model
history = model_cnn.fit(x_train, y_train,
                        batch_size=8,
                        epochs=35,
                        validation_data=(x_val, y_val),
                        verbose=1)

# Accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Evaluation
test_predictions = model_cnn.predict(x_test)
test_predictions = np.argmax(test_predictions, axis=1)
test_labels = np.argmax(y_test, axis=1)
test_accuracy = accuracy_score(test_labels, test_predictions)

print('Train Accuracy: {:.4f}'.format(train_accuracy[-1]))
print('Validation Accuracy: {:.4f}'.format(val_accuracy[-1]))
print('Test Accuracy: {:.4f}'.format(test_accuracy))


Epoch 1/35
239/239 [==============================] - 5s 18ms/step - loss: 2.5258 - accuracy: 0.2711 - val_loss: 2.3829 - val_accuracy: 0.3286
Epoch 2/35
239/239 [==============================] - 4s 15ms/step - loss: 2.4681 - accuracy: 0.2841 - val_loss: 2.3850 - val_accuracy: 0.3286
Epoch 3/35
239/239 [==============================] - 4s 15ms/step - loss: 2.4317 - accuracy: 0.2983 - val_loss: 2.3151 - val_accuracy: 0.3286
Epoch 4/35
239/239 [==============================] - 4s 15ms/step - loss: 2.4071 - accuracy: 0.2878 - val_loss: 2.2836 - val_accuracy: 0.3286
Epoch 5/35
239/239 [==============================] - 4s 15ms/step - loss: 2.3648 - accuracy: 0.2904 - val_loss: 2.2630 - val_accuracy: 0.3286
Epoch 6/35
239/239 [==============================] - 4s 15ms/step - loss: 2.3401 - accuracy: 0.2821 - val_loss: 2.2302 - val_accuracy: 0.3380
Epoch 7/35
239/239 [==============================] - 4s 15ms/step - loss: 2.2910 - accuracy: 0.3051 - val_loss: 2.2326 - val_accuracy: 0.3380

## U-Net

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# Define the model
model_fcn = Sequential()
model_fcn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model_fcn.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model_fcn.add(MaxPooling2D(pool_size=(2, 2)))
model_fcn.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_fcn.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_fcn.add(MaxPooling2D(pool_size=(2, 2)))
model_fcn.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_fcn.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_fcn.add(MaxPooling2D(pool_size=(2, 2)))
model_fcn.add(Flatten())
model_fcn.add(Dense(64, activation='relu'))
model_fcn.add(Dropout(0.3))
model_fcn.add(Dense(19, activation='softmax'))


model_fcn.compile(optimizer=Adam(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Data conversion
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Train the model
history = model_fcn.fit(x_train, y_train,
                        batch_size=8,
                        epochs=35,
                        validation_data=(x_val, y_val),
                        verbose=1)

# Accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
test_predictions = model_fcn.predict(x_test)
test_predictions = np.argmax(test_predictions, axis=1)
test_labels = np.argmax(y_test, axis=1)
test_accuracy = accuracy_score(test_labels, test_predictions)

print('Train Accuracy: {:.4f}'.format(train_accuracy[-1]))
print('Validation Accuracy: {:.4f}'.format(val_accuracy[-1]))
print('Test Accuracy: {:.4f}'.format(test_accuracy))


Epoch 1/35
239/239 [==============================] - 12s 37ms/step - loss: 2.4242 - accuracy: 0.2878 - val_loss: 2.1848 - val_accuracy: 0.3615
Epoch 2/35
239/239 [==============================] - 8s 32ms/step - loss: 2.2612 - accuracy: 0.3082 - val_loss: 2.3352 - val_accuracy: 0.3521
Epoch 3/35
239/239 [==============================] - 8s 32ms/step - loss: 2.2286 - accuracy: 0.3119 - val_loss: 2.1375 - val_accuracy: 0.3427
Epoch 4/35
239/239 [==============================] - 8s 32ms/step - loss: 2.1429 - accuracy: 0.3234 - val_loss: 2.1787 - val_accuracy: 0.3615
Epoch 5/35
239/239 [==============================] - 8s 32ms/step - loss: 2.0332 - accuracy: 0.3459 - val_loss: 2.1531 - val_accuracy: 0.3474
Epoch 6/35
239/239 [==============================] - 8s 32ms/step - loss: 1.8601 - accuracy: 0.3993 - val_loss: 2.2444 - val_accuracy: 0.3427
Epoch 7/35
239/239 [==============================] - 8s 32ms/step - loss: 1.6404 - accuracy: 0.4553 - val_loss: 2.2548 - val_accuracy: 0.417

## Restnet50

In [ ]:
import numpy as np
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score

# Initialization
base_model_resnet = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)


model_resnet = Sequential()
model_resnet.add(base_model_resnet)
model_resnet.add(Flatten())
model_resnet.add(Dense(64, activation='relu'))
model_resnet.add(Dropout(0.3))
model_resnet.add(Dense(19, activation='softmax'))

# Model
model_resnet.compile(
    optimizer=SGD(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Data conversion
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0

# Train the model
history = model_resnet.fit(
    x_train, y_train,
    batch_size=8,
    epochs=35,
    validation_data=(x_val, y_val),
    verbose=1
)

# Accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Evaluation
test_predictions = model_resnet.predict(x_test)
test_predictions = np.argmax(test_predictions, axis=1)
test_labels = np.argmax(y_test, axis=1)
test_accuracy = accuracy_score(test_labels, test_predictions)

print('Train Accuracy: {:.4f}'.format(train_accuracy[-1]))
print('Validation Accuracy: {:.4f}'.format(val_accuracy[-1]))
print('Test Accuracy: {:.4f}'.format(test_accuracy))


Epoch 1/35
239/239 [==============================] - 29s 96ms/step - loss: 2.0977 - accuracy: 0.4014 - val_loss: 23.3741 - val_accuracy: 0.1596
Epoch 2/35
239/239 [==============================] - 21s 87ms/step - loss: 1.5782 - accuracy: 0.5175 - val_loss: 7.8868 - val_accuracy: 0.1408
Epoch 3/35
239/239 [==============================] - 21s 87ms/step - loss: 1.3179 - accuracy: 0.5945 - val_loss: 3.8912 - val_accuracy: 0.2958
Epoch 4/35
239/239 [==============================] - 21s 87ms/step - loss: 1.0271 - accuracy: 0.6745 - val_loss: 2.2283 - val_accuracy: 0.4178
Epoch 5/35
239/239 [==============================] - 21s 88ms/step - loss: 0.7370 - accuracy: 0.7713 - val_loss: 1.6624 - val_accuracy: 0.5446
Epoch 6/35
239/239 [==============================] - 21s 89ms/step - loss: 0.5525 - accuracy: 0.8221 - val_loss: 1.5360 - val_accuracy: 0.5915
Epoch 7/35
239/239 [==============================] - 21s 89ms/step - loss: 0.4013 - accuracy: 0.8796 - val_loss: 1.4595 - val_accuracy

## VGG19

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


base_model = VGG19(include_top=False, input_shape=(256, 256, 3))
for layer in base_model.layers:
    layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
output = Dense(19, activation='softmax')(x)

# Model
model = Model(inputs=base_model.inputs, outputs=output)
opt = SGD(lr=0.0001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(featurewise_center=True, rotation_range=2, zoom_range=.2)
datagen.mean = [123.68, 116.779, 103.939]

# Data Conversion
x_train = x_train / 255.0
x_val = x_val / 255.0
x_test = x_test / 255.0


batch_size = 8
steps_per_epoch = len(x_train) // batch_size

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch,
                    epochs=40,
                    validation_data=(x_val, y_val),
                    verbose=1)

# Accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)

print('Train Accuracy: {:.4f}'.format(train_accuracy[-1]))
print('Validation Accuracy: {:.4f}'.format(val_accuracy[-1]))
print('Test Accuracy: {:.4f}'.format(test_accuracy))


D:\Anaconda\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/40
238/238 [==============================] - 23s 86ms/step - loss: 2.9470 - accuracy: 0.2780 - val_loss: 3.0237 - val_accuracy: 0.0939
Epoch 2/40
238/238 [==============================] - 17s 71ms/step - loss: 2.9118 - accuracy: 0.3127 - val_loss: 3.0011 - val_accuracy: 0.1033
Epoch 3/40
238/238 [==============================] - 17s 73ms/step - loss: 2.8892 - accuracy: 0.3142 - val_loss: 2.9793 - val_accuracy: 0.1268
Epoch 4/40
238/238 [==============================] - 18s 74ms/step - loss: 2.8679 - accuracy: 0.3121 - val_loss: 2.9581 - val_accuracy: 0.1315
Epoch 5/40
238/238 [==============================] - 18s 75ms/step - loss: 2.8464 - accuracy: 0.3142 - val_loss: 2.9376 - val_accuracy: 0.1408
Epoch 6/40
238/238 [==============================] - 19s 78ms/step - loss: 2.8257 - accuracy: 0.3142 - val_loss: 2.9175 - val_accuracy: 0.1362
Epoch 7/40
238/238 [==============================] - 18s 77ms/step - loss: 2.8065 - accuracy: 0.3127 - val_loss: 2.8980 - val_accuracy: